In [0]:
%sql

select * from anirvandecodes.silver.fact_visit;

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.table("anirvandecodes.silver.fact_visit")

In [0]:
w = (
    Window
        .partitionBy("patient_id")
        .orderBy("admission_date")
)

df_with_prev = (
    df.withColumn("previous_discharge", lag("discharge_date").over(w))
      .withColumn(
          "days_since_last_visit",
          datediff("admission_date", "previous_discharge")
      )
      .withColumn(
          "is_readmission_30d",
          when(col("days_since_last_visit") <= 30, 1).otherwise(0)
      )
)


In [0]:
display(df_with_prev)

In [0]:
gold_df = (
    df_with_prev.groupBy(
        "hospital_id",
        "hospital_name",
        "diagnosis_code"
    )
    .agg(
        count("*").alias("total_visits"),
        sum("is_readmission_30d").alias("total_readmissions"),
        round(sum("is_readmission_30d") / count("*"), 3).alias("readmission_rate"),
        sum("cost").alias("total_cost"),
        avg("cost").alias("avg_cost")
    )
    .withColumn("gold_load_timestamp", current_timestamp())
)


In [0]:
display(gold_df)



⭐ THIS GOLD TABLE ANSWERS THE CORE QUESTIONS YOU NEED FOR BI ⬇️
1. Which hospital has the highest readmission rate?
SELECT hospital_name, readmission_rate
FROM anirvandecodes.gold.hospital_disease_kpi
ORDER BY readmission_rate DESC;

2. Which disease category causes maximum readmissions?
SELECT diagnosis_category, SUM(total_readmissions) AS readm
FROM anirvandecodes.gold.hospital_disease_kpi
GROUP BY diagnosis_category
ORDER BY readm DESC;

3. Which hospital is performing worst for cardiac patients?
SELECT *
FROM anirvandecodes.gold.hospital_disease_kpi
WHERE diagnosis_category = 'Cardiology'
ORDER BY readmission_rate DESC;

4. Which hospital is spending the most on high-readmission diseases?
SELECT hospital_name, diagnosis_category, total_cost
FROM anirvandecodes.gold.hospital_disease_kpi
ORDER BY total_cost DESC;